Already exists:
- Boligrafica Twitter Tweepy
- Greenhouse Summon
- NOAA Data update GHD

Want to create:
- GFW - Datos Bolivia Deforestacion / Incendios
- Other twitter bots!
  - Mocking spongebob

Types of conditions:
- Runs on a schedule 
  - once every week (like boligrafica)
  - 4 times a day (like greenhouse summon)
- Runs if it has detected updated data (if new data, run)
  - Like NOAA data in GHD

In [189]:
import os
import sys
import pandas as pd
from datetime import datetime
from IPython import get_ipython

# environmental secrets when working locally
from dotenv import load_dotenv
load_dotenv()

True

In [190]:
os.getcwd()

'/home/daqub/Documents/GITHUB-UPDATE/update'

In [191]:
datetime.datetime(2022, 7, 7, 21, 39, 53, 887743)

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [128]:
f = datetime.datetime.now() - datetime.datetime(2022, 7, 7, 11, 39, 53, 887743)
f.total_seconds() / 60 /60 /24

0.42403457703703706

In [184]:
log = pd.read_csv("log.csv")
log

,,,,,,notebook_path,last_executed,execution_status
Greenhouse_Data/Twitter/GreenhouseSummon.ipynb,datetime.datetime(2022,7,7,22,14,41,612275),SUCCESS!


In [377]:
def SCRIPT_RUNNER(notebook_path, new_data = False, schedule_in_days = None):
    
    if schedule_in_days == None:
        EXECUTE = True
        
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("first time - creating dummy entry (year 2000)")
            # display(log)
            log.loc[notebook_path] = [datetime(2000, 7, 7, 21, 39), ""]
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        now = datetime.now()
        last_executed = datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # FINAL CHECK
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            

        
    print("Execute is: ", str(EXECUTE))
    if EXECUTE == True:

        
        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "SUCCESS!"
            print(status)
        except:
            status = "FAILURE."
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))   
        
        # log
        log.loc[notebook_path] = [datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful")
        
        return

####

SCRIPT_RUNNER(notebook_path = "Greenhouse_Data/Twitter/GreenhouseSummon.ipynb", schedule_in_days= 6/24)    

previous log found: 2022-07-07 22:36
difference in execution is: 0.004 days
needed difference to execute: 0.25 days
Execute is:  False


In [ ]:
# get_ipython().run_line_magic("run", "Greenhouse_Data/Twitter/GreenhouseSummon.ipynb")

In [ ]:
os.chdir("Greenhouse_Data/Twitter/")
try:
    %run GreenhouseSummon.ipynb
    os.chdir("../..")
except:
    pass
    os.chdir("../..")

In [ ]:
# NOW = datetime.now()
# NOW

In [ ]:
print("success yeah!")